In [4]:
import numpy as np
import pandas as pd
import optuna
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

# Step 1: Load the data
tesla_data = pd.read_csv('Get_data/Tesla_Close_2019_2024.csv', parse_dates=['Date'], index_col='Date')
sp500_data = pd.read_csv('Get_data/S&P500_Close_2019_2024.csv', parse_dates=['Date'], index_col='Date')
ixic_data = pd.read_csv('Get_data/IXIC_Close_2019_2024.csv', parse_dates=['Date'], index_col='Date')

# Merge datasets on 'Date'
merged_data = tesla_data.merge(sp500_data, left_index=True, right_index=True, suffixes=('', '_sp500'))
merged_data = merged_data.merge(ixic_data, left_index=True, right_index=True, suffixes=('', '_ixic'))

# Step 2: Preprocess the data
data = merged_data[['Close', 'Close_sp500', 'Close_ixic']]
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)

# Prepare the dataset for LSTM
def create_dataset(dataset, time_step=60):
    X, y = [], []
    for i in range(time_step, len(dataset)):
        X.append(dataset[i - time_step:i])
        y.append(dataset[i, 0])  # Predict the 'Close' price of Tesla
    return np.array(X), np.array(y)

time_step = 60
X, y = create_dataset(scaled_data, time_step)

# Split data into training and test sets
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Function to create and train the LSTM model with Optuna
def objective(trial):
    # Hyperparameters to optimize
    lstm_units = trial.suggest_int('lstm_units', 10, 100)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64])
    epochs = trial.suggest_int('epochs', 10, 100)

    # Build the LSTM model
    model = Sequential([
        Input(shape=(time_step, X_train.shape[2])),
        LSTM(lstm_units, return_sequences=True),
        Dropout(dropout_rate),
        LSTM(lstm_units),
        Dropout(dropout_rate),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mean_squared_error')

    # Train the model
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)

    # Evaluate the model
    test_predictions = model.predict(X_test)
    test_predictions_rescaled = scaler.inverse_transform(
        np.hstack([test_predictions, X_test[:, -1, 1:]])
    )[:, 0]
    y_test_rescaled = scaler.inverse_transform(
        np.hstack([y_test.reshape(-1, 1), X_test[:, -1, 1:]])
    )[:, 0]

    rmse = np.sqrt(mean_squared_error(y_test_rescaled, test_predictions_rescaled))
    return rmse

# Run the optimization
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

# Print the best hyperparameters
print('Best trial:')
trial = study.best_trial
print(f'  Value (RMSE): {trial.value}')
print('  Params:')
for key, value in trial.params.items():
    print(f'    {key}: {value}')


D:\Python\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2024-11-16 16:45:27,627] A new study created in memory with name: no-name-3d7f2c11-8bb3-4bcd-a644-0788d78c19bc


9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step


[I 2024-11-16 16:45:41,463] Trial 0 finished with value: 16.484965196435454 and parameters: {'lstm_units': 40, 'dropout_rate': 0.36096636626618184, 'learning_rate': 0.0008182358133305553, 'batch_size': 32, 'epochs': 13}. Best is trial 0 with value: 16.484965196435454.


9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step


[I 2024-11-16 16:46:07,472] Trial 1 finished with value: 15.064377760387416 and parameters: {'lstm_units': 34, 'dropout_rate': 0.4165979108117278, 'learning_rate': 0.002084822690787443, 'batch_size': 64, 'epochs': 46}. Best is trial 1 with value: 15.064377760387416.


...

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step


[I 2024-11-16 17:48:56,432] Trial 48 finished with value: 9.092727075753626 and parameters: {'lstm_units': 74, 'dropout_rate': 0.2031218442375483, 'learning_rate': 0.002358195463295874, 'batch_size': 32, 'epochs': 74}. Best is trial 12 with value: 8.509432236765758.


9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step


[I 2024-11-16 17:49:18,828] Trial 49 finished with value: 13.878202394047674 and parameters: {'lstm_units': 10, 'dropout_rate': 0.24420458011118884, 'learning_rate': 0.0017260536037147054, 'batch_size': 32, 'epochs': 47}. Best is trial 12 with value: 8.509432236765758.


Best trial:
  Value (RMSE): 8.509432236765758
  Params:
    lstm_units: 53
    dropout_rate: 0.2692443564327529
    learning_rate: 0.001099003730699956
    batch_size: 16
    epochs: 99


In [6]:
import numpy as np
import pandas as pd
import optuna
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

# Step 1: Load the data
tesla_data = pd.read_csv('Get_data/Tesla_Close_2019_2024.csv', parse_dates=['Date'], index_col='Date')
sp500_data = pd.read_csv('Get_data/S&P500_Close_2019_2024.csv', parse_dates=['Date'], index_col='Date')
ixic_data = pd.read_csv('Get_data/IXIC_Close_2019_2024.csv', parse_dates=['Date'], index_col='Date')

# Merge datasets on 'Date'
merged_data = tesla_data.merge(sp500_data, left_index=True, right_index=True, suffixes=('', '_sp500'))
merged_data = merged_data.merge(ixic_data, left_index=True, right_index=True, suffixes=('', '_ixic'))

# Step 2: Preprocess the data
data = merged_data[['Close', 'Close_sp500', 'Close_ixic']]
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)

# Prepare the dataset for LSTM
def create_dataset(dataset, time_step=60):
    X, y = [], []
    for i in range(time_step, len(dataset)):
        X.append(dataset[i - time_step:i])
        y.append(dataset[i, 0])  # Predict the 'Close' price of Tesla
    return np.array(X), np.array(y)

time_step = 60
X, y = create_dataset(scaled_data, time_step)

# Split data into training and test sets
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Function to create and train the LSTM model with Optuna
def objective(trial):
    # Hyperparameters to optimize
    lstm_units = trial.suggest_int('lstm_units', 55, 60)
    dropout_rate = trial.suggest_float('dropout_rate', 0.265, 0.275)
    learning_rate = trial.suggest_float('learning_rate', 0.00095, 0.0011, log=True)
    batch_size = trial.suggest_categorical('batch_size', [16])
    epochs = trial.suggest_int('epochs', 100, 200)

    # Build the LSTM model
    model = Sequential([
        Input(shape=(time_step, X_train.shape[2])),
        LSTM(lstm_units, return_sequences=True),
        Dropout(dropout_rate),
        LSTM(lstm_units),
        Dropout(dropout_rate),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mean_squared_error')

    # Train the model
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)

    # Evaluate the model
    test_predictions = model.predict(X_test)
    test_predictions_rescaled = scaler.inverse_transform(
        np.hstack([test_predictions, X_test[:, -1, 1:]])
    )[:, 0]
    y_test_rescaled = scaler.inverse_transform(
        np.hstack([y_test.reshape(-1, 1), X_test[:, -1, 1:]])
    )[:, 0]

    rmse = np.sqrt(mean_squared_error(y_test_rescaled, test_predictions_rescaled))
    return rmse

# Run the optimization
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)

# Print the best hyperparameters
print('Best trial:')
trial = study.best_trial
print(f'  Value (RMSE): {trial.value}')
print('  Params:')
for key, value in trial.params.items():
    print(f'    {key}: {value}')


[I 2024-11-16 18:32:36,499] A new study created in memory with name: no-name-f93d06e9-52b2-4158-a83c-ff44a9833529


9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step


[I 2024-11-16 18:34:22,601] Trial 0 finished with value: 16.716076220282748 and parameters: {'lstm_units': 55, 'dropout_rate': 0.27186907741824035, 'learning_rate': 0.0010704914582791985, 'batch_size': 16, 'epochs': 100}. Best is trial 0 with value: 16.716076220282748.


9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step


[I 2024-11-16 18:37:46,260] Trial 1 finished with value: 9.224599301221293 and parameters: {'lstm_units': 60, 'dropout_rate': 0.26640706444585577, 'learning_rate': 0.0010772910183478501, 'batch_size': 16, 'epochs': 191}. Best is trial 1 with value: 9.224599301221293.


...

9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step


[I 2024-11-16 18:58:55,508] Trial 8 finished with value: 11.239318889479655 and parameters: {'lstm_units': 56, 'dropout_rate': 0.2719894914803458, 'learning_rate': 0.0010327075293807089, 'batch_size': 16, 'epochs': 157}. Best is trial 2 with value: 8.02193333478337.


9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step


[I 2024-11-16 19:01:39,861] Trial 9 finished with value: 11.089835807708328 and parameters: {'lstm_units': 55, 'dropout_rate': 0.2725608737666196, 'learning_rate': 0.0010079857364463669, 'batch_size': 16, 'epochs': 152}. Best is trial 2 with value: 8.02193333478337.


Best trial:
  Value (RMSE): 8.02193333478337
  Params:
    lstm_units: 57
    dropout_rate: 0.2714727016483041
    learning_rate: 0.0010820784404161896
    batch_size: 16
    epochs: 128
